In [1]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOllama
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage, HumanMessage
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [14]:


from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.chat_models import ChatOllama
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough


embedding_model = OllamaEmbeddings(model="nomic-embed-text")
CHROMA_DB_PATH = r"C:\Users\suram\OneDrive\Desktop\rag project\chroma_db"

\
vectorstore = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embedding_model)


memory = ConversationBufferWindowMemory(k=5, return_messages=True)


system_message = SystemMessage(content=(
    "You are a legal assistant specializing in the Indian Penal Code (IPC). "
    "When a user asks about an IPC section, provide: "
    "(1) Details of the section, (2) Any related or defendable sections, "
    "(3) The type of law it falls under (criminal, procedural, etc.). "
    "If the user provides an IPC section number, directly return its details, "
    "defendable sections, and classification. Ensure responses are clear, "
    "concise, and legally accurate."
))


prompt = PromptTemplate(
    input_variables=["history", "retrieved_context", "query"],
    template=(
        "The following is a conversation between a user and a legal AI assistant:\n"
        "{history}\n\n"
        "Retrieved Legal Context:\n{retrieved_context}\n\n"
        "User: {query}\n"
        "AI:"
    )
)


retriever = (
    RunnablePassthrough()
    | (lambda query: embedding_model.embed_query(query))
    | (lambda query_embedding: vectorstore.similarity_search_by_vector(query_embedding, k=3))
    | (lambda docs: "\n".join([doc.page_content for doc in docs]) if docs else "No relevant legal information found.")
)


llm_chain = LLMChain(
    llm=ChatOllama(model="mistral"),
    prompt=prompt
)


chat_pipeline = (
    RunnablePassthrough()
    | (lambda query: {
        "history": "\n".join(
            [f"{msg.type.capitalize()}: {msg.content}" for msg in ([system_message] + memory.load_memory_variables({}).get("history", []))]
        ),
        "retrieved_context": retriever.invoke(query),
        "query": query
    })
    | llm_chain
    | (lambda response: response["text"] if isinstance(response, dict) and "text" in response else str(response))
)


def retrieve_and_answer(query):
    """Retrieve relevant legal information and generate an AI response."""
    
    
    retrieved_context = retriever.invoke(query)

    
    previous_messages = memory.load_memory_variables({}).get("history", [])
    
    
    formatted_history = "\n".join(
        [f"{msg.type.capitalize()}: {msg.content}" for msg in ([system_message] + previous_messages)]
    )

    
    response = chat_pipeline.invoke({
        "history": formatted_history,
        "retrieved_context": retrieved_context,
        "query": query
    })

    
    memory.save_context({"input": query}, {"output": response})
    
    return response


if __name__ == "__main__":
    print("Legal AI Chatbot (Type 'exit' or 'quit' to stop)\n")
    
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Exiting chatbot. Goodbye!")
            break
        
        ai_response = retrieve_and_answer(user_input)
        print(f"AI: {ai_response}\n")


Legal AI Chatbot (Type 'exit' or 'quit' to stop)

AI: 1. I'm an assistant specializing in the Indian Penal Code (IPC). The section you provided is not a specific IPC section number but rather a part of the Indian Evidence Act and some amendments related to it.
    - Details: This text describes various terms used in the context of criminal proceedings, like "complainant," "High Court," "inquiry," "investigation," etc., and an amendment made to section 53A of the Indian Evidence Act. Additionally, it mentions section 54 (Previous bad character not relevant, except in reply) and section 39 (Public to give information of certain offences).
    - Related or defendable sections: Sections mentioned under section 39 of the Indian Evidence Act are related to offenses punishable under various sections of the IPC.
    - Type of law: This text falls under Criminal Procedure Law and Evidence Law.

AI: 1. The IPC section for murder is Section 302 - Murder.
   2. Related or defendable sections inclu